In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [4]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()
print(cur_path)

# Retrieving the images and their labels
for i in range(classes):
    path = os.path.join(cur_path, 'train', str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\' + a)
            image = image.resize((30, 30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

# Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)

e:\Downloads\zad\obrazy\dataProcesing
Error loading image
(39208, 30, 30, 3) (39208,)


In [5]:
# Splitting training and testing dataset
X_t1, X_t2, y_t1, y_t2 = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_t1.shape, X_t2.shape, y_t1.shape, y_t2.shape)
#Converting the labels into one hot encoding
y_t1 = to_categorical(y_t1, 43)
y_t2 = to_categorical(y_t2, 43)

print(f"Shape = {X_t1.shape[1:]}")

(31366, 30, 30, 3) (7842, 30, 30, 3) (31366,) (7842,)
(30, 30)
Shape = (30, 30, 3)


In [7]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_t1.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))
#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
eps = 15

In [18]:
# for layer in model.layers:
#     for output in layer.input_shape:
#         print(output)
print(model.layers[0].input_shape)

(None, 30, 30, 3)


In [5]:
anc = model.fit(X_t1, y_t1, batch_size=32, epochs=eps, validation_data=(X_t2, y_t2))
model.save("roadSignsModel.h5")

Epoch 1/15
981/981 [==============================] - 61s 61ms/step - loss: 2.0284 - accuracy: 0.4891 - val_loss: 0.4983 - val_accuracy: 0.8653
Epoch 2/15
981/981 [==============================] - 63s 65ms/step - loss: 0.7349 - accuracy: 0.7822 - val_loss: 0.2632 - val_accuracy: 0.9263
Epoch 3/15
981/981 [==============================] - 56s 57ms/step - loss: 0.5315 - accuracy: 0.8446 - val_loss: 0.2543 - val_accuracy: 0.9296
Epoch 4/15
981/981 [==============================] - 62s 64ms/step - loss: 0.4254 - accuracy: 0.8731 - val_loss: 0.1223 - val_accuracy: 0.9652
Epoch 5/15
981/981 [==============================] - 52s 53ms/step - loss: 0.3720 - accuracy: 0.8904 - val_loss: 0.1326 - val_accuracy: 0.9614
Epoch 6/15
981/981 [==============================] - 54s 55ms/step - loss: 0.3334 - accuracy: 0.9004 - val_loss: 0.1189 - val_accuracy: 0.9666
Epoch 7/15
981/981 [==============================] - 55s 56ms/step - loss: 0.3161 - accuracy: 0.9059 - val_loss: 0.0841 - val_accuracy:

e:\Downloads\zad\obrazy\ImageVenv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Accuracy with the test data
y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
   image = Image.open(img)
   image = image.resize((30,30))
   data.append(np.array(image))
X_test=np.array(data)
predictions = model.predict(X_test)

# Get the predicted class indices
pred_classes = np.argmax(predictions, axis=1)

# Calculate accuracy for all samples
accuracy = accuracy_score(labels, pred_classes)
print("Accuracy:", accuracy)

395/395 [==============================] - 4s 11ms/step
Accuracy: 0.9296912114014252
